In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, sum, count, max, min
from azure.storage.blob import BlobServiceClient
import os

silver_path = f"{os.environ["AZURE_STORAGE_ACCOUNT_URL"]}/silver"
gold_path = f"{os.environ["AZURE_STORAGE_ACCOUNT_URL"]}/gold"

# Inicialização do SparkSession
spark = SparkSession.builder \
    .appName("Transformar Silver para Gold") \
    .getOrCreate()

# Listar as tabelas na camada Silver (simulação de listar arquivos parquet)
silver_tables = [f"{silver_path}/{file}" for file in ["tabela1.parquet", "tabela2.parquet"]]  # Ajuste para seu ambiente

# Exemplo de transformações específicas para cada tabela
for table_path in silver_tables:
    # Nome da tabela com base no caminho do arquivo
    table_name = table_path.split("/")[-1].replace(".parquet", "")

    # Leitura dos dados da camada Silver
    df_silver = spark.read.parquet(table_path)

    # Exemplos de transformações para camada Gold:
    if table_name == "tabela1":
        # Exemplo 1: Agregação - Calcular média e total de vendas por categoria
        df_gold = df_silver.groupBy("categoria") \
            .agg(
                count("*").alias("total_registros"),
                sum("valor_venda").alias("soma_vendas"),
                avg("valor_venda").alias("media_vendas")
            )

    elif table_name == "tabela2":
        # Exemplo 2: Adicionar métricas calculadas - Margem de lucro
        df_gold = df_silver \
            .withColumn("lucro", col("valor_venda") - col("custo")) \
            .withColumn("margem", (col("lucro") / col("valor_venda")) * 100)

    else:
        # Exemplo 3: Análise temporal - Obter máximos e mínimos por ano
        df_gold = df_silver.groupBy("ano") \
            .agg(
                max("valor_venda").alias("max_venda"),
                min("valor_venda").alias("min_venda")
            )

    # Caminho da tabela transformada na camada Gold
    gold_table_path = f"{gold_path}/{table_name}_gold.parquet"

    # Gravação no Azure Data Lake em formato Parquet
    df_gold.write \
        .mode("overwrite") \
        .parquet(gold_table_path)

    print(f"Tabela {table_name} transformada e salva em {gold_table_path}")

# Encerrar sessão Spark
spark.stop()
